# Глубокие нейронные сети для классификации изображений

Вы будете использовать функции, которые вы реализовали в предыдущем задании, для построения глубокой сети и применять ее для классификации cat против non-cat. Вы увидите улучшение точности предсказания по сравнению с предыдущей реализацией логистической регрессии.

**В рамках данной лабораторной работы будут приобретены следующие навыки:**
- Создание и применение глубокой нейронной сети для обучения с учителем. 

`Данный материал опирается и использует материалы курса Deep Learning от организации deeplearning.ai`
 
 Ссылка на осной курс (для желающих получить сертификаты): https://www.coursera.org/specializations/deep-learning

## 1 - Пакеты/Библиотеки ##

Первоначально необходимо запустить ячейку ниже, чтобы импортировать все пакеты, которые вам понадобятся во время лабораторной работы.
- [numpy](www.numpy.org) является основным пакетом для научных вычислений в Python.
- [h5py](http://www.h5py.org) это общий пакет для взаимодействия с набором данных, которые хранятся в файле H5.
- [matplotlib](http://matplotlib.org) это пакет для отрисовки графиков в Python.
- [PIL](http://www.pythonware.com/products/pil/) и [scipy](https://www.scipy.org/) используются здесь, чтобы проверить построенную модель с собственным (загруженным) изображением в конце лабораторной работы.
- dnn_app_utils функции реализованные в предыдущей лабораторной работе.

In [ ]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from dnn_app_utils_v2 import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

## 2 - Датасет

**Постановка задачи**: Дан набор данных ("data.h5") содержащий:
    - обучающий набор m_train изображений рамеченных как cat (y=1) или non-cat (y=0);
    - тестовый набор m_test изображений рамеченных как cat (y=1) или non-cat (y=0);
    - каждое изображение имеет размер (num_px, num_px, 3) где 3 это 3 канала (RGB). Таким образом, каждое изображение это квадрат, где (height = num_px) и (width = num_px).

Требуется построить прострой алгоритм, который сможет классифицировать объект на изображении (cat это или нет).

Для того, чтобы познакомиться с набором данных, загрузите блок с кодом ниже.

In [ ]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

In [ ]:
index = 24
plt.imshow(train_x_orig[index])
print ("y = " + str(train_y[0,index]) + ". It's a " + classes[train_y[0,index]].decode("utf-8") +  " picture.")

In [ ]:
m_train = train_x_orig.shape[0]
num_px = train_x_orig.shape[1]
m_test = test_x_orig.shape[0]

print ("Количество обучающих примеров: " + str(m_train))
print ("Количество тестовых примеров: " + str(m_test))
print ("Каждое изображение имеет размер: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_x_orig shape: " + str(train_x_orig.shape))
print ("train_y shape: " + str(train_y.shape))
print ("test_x_orig shape: " + str(test_x_orig.shape))
print ("test_y shape: " + str(test_y.shape))

Изменение формы и стандартизация изображения перед подачей их в сеть. Код приведен в ячейке ниже.

<img src="images/imvectorkiank.png" style="width:450px;height:300px;">

<caption><center> <u>Рисунок 1</u> <br> </center></caption>

In [ ]:
# Изменение размера обучающего и тестового множества примеров
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T   # The "-1" makes reshape flatten the remaining dimensions
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

# Стандартизация
train_x = train_x_flatten/255.
test_x = test_x_flatten/255.

print ("train_x's shape: " + str(train_x.shape))
print ("test_x's shape: " + str(test_x.shape))


$12,288$ = $64 \times 64 \times 3$ размер вектора изображения.

## 3 - Архитектура модели

В лабораторной работе необходимо построить 2 модели:
- 2-слойнная нейронная сеть
- L-слойнная глубокая нейронная сеть

Затем необходимо будет сравнить производительность моделей, а также попробовать различные значения для $L$. 

### 3.1 - 2-слойнная нейронная сеть

<img src="images/2layerNN_kiank.png" style="width:650px;height:400px;">
<caption><center> <u>Рисунок 2</u>: 2-слойнная нейронная сеть. <br> Модель выглядит вот так: ***INPUT -> LINEAR -> RELU -> LINEAR -> SIGMOID -> OUTPUT***. </center></caption>

<u>Детализация архитектуры на Рисунке 2</u>:
- Вход (64,64,3) изображение преобразованное в вектор размером $(12288,1)$. 
- Данный вектор: $[x_0,x_1,...,x_{12287}]^T$ затем умножается на матрицу весов $W^{[1]}$ размером $(n^{[1]}, 12288)$.
- Затем добавляется смещение и вычисляете функцию активации, чтобы получить следующий вектор: $[a_0^{[1]}, a_1^{[1]},..., a_{n^{[1]}-1}^{[1]}]^T$.
- Затем необходимо повторить данный процесс.
- Результирующий вектор предыдущего слоя умножается на матрицу весов $W^{[2]}$ и добавляется смещение (bias). 
- В итоге, вычисляется sigmoid по выходу предыдущего слоя. Если значение больше 0.5, то изображение классифицируется как cat.

### 3.2 - L-слойнаая глубокая нейронная сеть

<img src="images/LlayerNN_kiank.png" style="width:650px;height:400px;">
<caption><center> <u>Рисунок 3</u>: L-слойнаая глубокая нейронная сеть. <br> Модель выглядит вот так: ***[LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID***</center></caption>

<u>Детализация архитектуры на Рисунке 3</u>:
- Вход (64,64,3) изображение преобразованное в вектор размером $(12288,1)$. 
- Данный вектор: $[x_0,x_1,...,x_{12287}]^T$ затем умножается на матрицу весов $W^{[1]}$ размером $(n^{[1]}, 12288)$ и добавляется смещение $b^{[1]}$. Результат называется линейной единицей измерения.
- Далее производится расчёт функции активации RELU для линейного блока. Этот процесс можно повторить несколько раз для каждого слоя $(W^{[l]}, b^{[l]})$.
- В итоге, вычисляется sigmoid по выходу предыдущего слоя. Если значение больше 0.5, то изображение классифицируется как cat.

### 3.3 - Обобщённая методология
Методика для построения модели глубокого обучения:
    1. Инициализировать параметры
    2. Цикл для num_iterations:
        a. прямое распространение
        б. Функция вычисления стоимости 
        c. обратное распространение
        д. Параметры обновления (используя параметры, и градиенты из сеть с обратным распространением ошибки) 
    4. Используйте обученные параметры для прогнозирования

## 4 - 2-слойнная нейронная сеть

**Упражнение**: Используйте функции построенные в предыдущей лабораторной работе для построения 2-слойнной нейронной сети со следующей структурой: *LINEAR -> RELU -> LINEAR -> SIGMOID*. Функции, которые вам могут понадобиться, и их аргументы:
```python
def initialize_parameters(n_x, n_h, n_y):
    ...
    return parameters 
def linear_activation_forward(A_prev, W, b, activation):
    ...
    return A, cache
def compute_cost(AL, Y):
    ...
    return cost
def linear_activation_backward(dA, cache, activation):
    ...
    return dA_prev, dW, db
def update_parameters(parameters, grads, learning_rate):
    ...
    return parameters
```

In [ ]:
### Константы для построения модели ####
n_x = 12288 # num_px * num_px * 3
n_h = 7
n_y = 1
layers_dims = (n_x, n_h, n_y)

In [ ]:
# ОЦЕНИВАЕМОЕ: two_layer_model

def two_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    """
    Реализация 2-х слойнной нейронной сети: LINEAR->RELU->LINEAR->SIGMOID.
    
    Arguments:
    X -- матрица признаков (n_x, number of examples)
    Y -- вектор меток (0 - cat, 1 - non-cat), размером (1, number of examples)
    layers_dims -- размерность нейронной сети (n_x, n_h, n_y)
    num_iterations -- количество итераций градиентного спуска
    learning_rate -- скорость сходимости градиентного спуска
    print_cost -- печать каждые 100 шагов
    
    Returns:
    parameters -- словарь с параметрами W1, W2, b1, и b2
    """
    
    np.random.seed(1)
    grads = {}
    costs = []  
    m = X.shape[1] 
    (n_x, n_h, n_y) = layers_dims
    
    # Инициализируйте словарь параметров, вызвав одну из ранее реализованных функций
    ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (≈ 1 line of code)
    parameters = None
    ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    for i in range(0, num_iterations):

        # Прямое распространение: LINEAR -> RELU -> LINEAR -> SIGMOID. Вход: "X, W1, b1". Выход: "A1, cache1, A2, cache2".
        ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (≈ 2 строки кода)
        A1, cache1 = None
        A2, cache2 = None
        ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
        
        # Расчёт потерь
        ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (≈ 1 line of code)
        cost = None
        ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
        
        # Запуска алгоритма обратного распространения
        dA2 = - (np.divide(Y, A2) - np.divide(1 - Y, 1 - A2))
        
        # Обратное распространение. Вход: "dA2, cache2, cache1". Выход: "dA1, dW2, db2; dA0, dW1, db1".
        ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (≈ 2 строки кода)
        dA1, dW2, db2 = None
        dA0, dW1, db1 = None
        ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
        
        grads['dW1'] = dW1
        grads['db1'] = db1
        grads['dW2'] = dW2
        grads['db2'] = db2
        
        # Обновление параметров.
        ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (approx. 1 line of code)
        parameters = None
        ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###

        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]
        
        if print_cost and i % 100 == 0:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if print_cost and i % 100 == 0:
            costs.append(cost)
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

Запустите ячейку ниже, чтобы обучить параметры. Проверьте, работает ли модель. Значение функции потерь должно уменьшаться. Для выполнения 2500 итераций может потребоваться до 5 минут. 

In [ ]:
parameters = two_layer_model(train_x, train_y, layers_dims = (n_x, n_h, n_y), num_iterations = 2500, print_cost=True)

**Ожидаемый результат**:
<table> 
    <tr>
        <td> **Cost after iteration 0**</td>
        <td> 0.6930497356599888 </td>
    </tr>
    <tr>
        <td> **Cost after iteration 100**</td>
        <td> 0.6464320953428849 </td>
    </tr>
    <tr>
        <td> **...**</td>
        <td> ... </td>
    </tr>
    <tr>
        <td> **Cost after iteration 2400**</td>
        <td> 0.048554785628770206 </td>
    </tr>
</table>

Теперь можно использовать обученные параметры для классификации изображений из набора данных. Чтобы просмотреть прогнозы по наборам тренировок и тестов, запустите ячейку ниже.

In [ ]:
predictions_train = predict(train_x, train_y, parameters)

**Ожидаемый результат**:
<table> 
    <tr>
        <td> **Accuracy**</td>
        <td> 1.0 </td>
    </tr>
</table>

In [ ]:
predictions_test = predict(test_x, test_y, parameters)

**Ожидаемый результат**:

<table> 
    <tr>
        <td> **Accuracy**</td>
        <td> 0.72 </td>
    </tr>
</table>

**Вывод**: Можно заметить, что выполнение модели на меньшем количестве итераций (например, 1500) дает лучшую точность на тестовом наборе. Это называется "ранняя остановка". Ранняя остановка-это способ предотвратить переобучение.

2-слойная нейронная сеть имеет лучшую точность (72%), чем реализация логистической регрессии (70%).

## 5 - L-слойнаая глубокая нейронная сеть

**Упражнение**: Используйте функции построенные в предыдущей лабораторной работе для построения $L$-слойнной нейронной сети со следующей структурой: *[LINEAR -> RELU]$\times$(L-1) -> LINEAR -> SIGMOID*. Функции, которые вам могут понадобиться, и их аргументы:
```python
def initialize_parameters_deep(layer_dims):
    ...
    return parameters 
def L_model_forward(X, parameters):
    ...
    return AL, caches
def compute_cost(AL, Y):
    ...
    return cost
def L_model_backward(AL, Y, caches):
    ...
    return grads
def update_parameters(parameters, grads, learning_rate):
    ...
    return parameters
```

In [ ]:
### Константы ###
layers_dims = [12288, 20, 7, 5, 1] #  5-слойнная модель

In [ ]:
# ОЦЕНИВАЕМОЕ: L_layer_model

def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):#lr was 0.009
    """
    Реализация L-х слойнной нейронной сети: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- матрица признаков (n_x, number of examples)
    Y -- вектор меток (0 - cat, 1 - non-cat), размером (1, number of examples)
    layers_dims -- размерность нейронной сети (number of layers + 1)
    num_iterations -- количество итераций градиентного спуска
    learning_rate -- скорость сходимости градиентного спуска
    print_cost -- печать каждые 100 шагов
    
    Returns:
    parameters -- параметры модели, используемые для предсказания.
    """

    np.random.seed(1)
    costs = []
    
    # Инициализация параметров.
    ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ###
    parameters = None
    ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
    
    for i in range(0, num_iterations):

        # Прямое распространение: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (≈ 1 строка кода)
        AL, caches = None
        ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
        
        # Расчёт потерь.
        ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (≈ 1 строка кода)
        cost = None
        ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
    
        # Обратное распространение.
        ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (≈ 1 строка кода)
        grads = None
        ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
 
        # Обновление параметров.
        ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (≈ 1 строка кода)
        parameters = None
        ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
                
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

Запустите ячейку ниже, чтобы обучить параметры. Проверьте, работает ли модель. Значение функции потерь должно уменьшаться. Для выполнения 2500 итераций может потребоваться до 5 минут. 

In [ ]:
parameters = L_layer_model(train_x, train_y, layers_dims, num_iterations = 2500, print_cost = True)

**Ожидаемый результат**:
<table> 
    <tr>
        <td> **Cost after iteration 0**</td>
        <td> 0.771749 </td>
    </tr>
    <tr>
        <td> **Cost after iteration 100**</td>
        <td> 0.672053 </td>
    </tr>
    <tr>
        <td> **...**</td>
        <td> ... </td>
    </tr>
    <tr>
        <td> **Cost after iteration 2400**</td>
        <td> 0.092878 </td>
    </tr>
</table>

In [ ]:
pred_train = predict(train_x, train_y, parameters)

<table>
    <tr>
    <td>
    **Train Accuracy**
    </td>
    <td>
    0.985645933014
    </td>
    </tr>
</table>

In [ ]:
pred_test = predict(test_x, test_y, parameters)

**Ожидаемый результат**:

<table> 
    <tr>
        <td> **Test Accuracy**</td>
        <td> 0.8 </td>
    </tr>
</table>

5-слойная нейронная сеть имеет лучшую точность (80%), чем реализация 2-слойнной нейронной сети (72%).

##  6 - Аналих результатов ##

Во-первых, давайте взглянем на некоторые изображения, которые Модель L-слоями разметила неверно.

In [ ]:
print_mislabeled_images(classes, test_x, test_y, pred_test)

**Несколько типов изображений, которые модель плохо классифицирует:** 
- Тело кошки в необычном положении
- Кошка появляется на фоне аналогичного цвета как и кошка
- Необычный окрас и вид кошки
- Яркость изображения
- Изменение масштаба (кошка очень большая или маленькая на изображении)


## 7 - Тестирование с вашими изображениями ##

Вы можете использовать собственные изображения для тестирования разработанной модели. Что необходимо сделать:
    1. Загрузите своё изображение в директорию /image/;
    2. В коде ниже подставьте наименование изображения, соответствующее вашему;
    3. Запустите алгоритм ниже (1 = cat, 0 = non-cat).

In [ ]:
## НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ## (вставьте своё изображение) 
my_image = "my_image.jpg" # измените на имя своего изображения
my_label_y = [1] # истинный класс изображения (1 -> cat, 0 -> non-cat)
## ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ##

fname = "images/" + my_image
image = np.array(ndimage.imread(fname, flatten=False))
my_image = scipy.misc.imresize(image, size=(num_px,num_px)).reshape((num_px*num_px*3,1))
my_predicted_image = predict(my_image, my_label_y, parameters)

plt.imshow(image)
print ("y = " + str(np.squeeze(my_predicted_image)) + ", модель предсказала \"" + classes[int(np.squeeze(my_predicted_image)),].decode("utf-8") +  "\" изображение.")

**Используемый материал:**:
- Курс Deep Learning; https://www.coursera.org/specializations/deep-learning
- for auto-reloading external module: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython